In [3]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score

import pandas as pd
import numpy as np
import random
import time
import datetime

### 데이터 로드

In [4]:
df = pd.read_csv('data_whole.csv', encoding='utf-8')

In [5]:
df = df.dropna(axis=0)

In [6]:
sentences = []
for title, body in zip(df['Headline'], df['Content']):
    total = title + '[SEP]' + body
    sentences.append(total)

labels = df['Class'].values

In [7]:
train_data, validation_data, train_labels, validation_labels = train_test_split(sentences, labels,
                                                    random_state = 2018,
                                                    test_size=0.1)

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

def tokenize_sentences(tokenizer, sentences, max_length):
    tokenized_sentences = tokenizer(
        sentences,
        return_tensors = "pt",
        padding = True,
        truncation = True,
        max_length = max_length,
        add_special_tokens = True,
        return_token_type_ids = False,
    )
    return tokenized_sentences

print(train_data[0])
print(tokenizer.tokenize(train_data[0]))
tokenized_text = tokenize_sentences(tokenizer, train_data[0], 450)
print(tokenized_text['input_ids'])
print(tokenized_text['attention_mask'])

제멋대로 비급여 진료비, 비교하기 쉽게 재정비[SEP]의료기관마다 제멋대로였던 비급여진료비 정보가 재정비된다.
보건복지부는 의료기관의 비급여 비용 고지 방법을 구체적으로 제시함으로써 국민들이 의료기관 이용시 비급여 비용을 보다 알기 쉽고, 찾기 쉽도록 비급여 진료비용 고지 지침을 개정한다고 밝혔다.
그동안 국민의 알 권리 보장을 위해 비급여 가격을 기재한 책자 등을 의료기관에 비치하고 인터넷 홈페이지에 게시하도록 했다.
그러나 의료기관이 자율적인 방식으로 고지하다 보니 기관마다 용어와 분류 방식이 달랐다.
이때문에 국민들이 이해하고 가격 비교하기에 어려움이 있었다.
또 기관 내 비치 장소나 홈페이지 내 위치를 찾기 어렵다는 지적이 있어 왔다.
이에 비급여 항목 용어와 분류체계를 따르도록 표준화하고 구체적인 양식을 제공함으로써 의료기관이 쉽게 고지할 수 있고 국민들도 쉽게 찾을 수 있도록 했다.
개정안에 따르면 전체 비급여 비용을 행위료, 치료재료대, 약제비, 제증명수수료, 선택진료료의 5분야로 분류했다.
시술료, 검사료 등 행위료는 치료재료와 약제가 포함되는 경우가 많아 치료재료와 약제비 포함 여부를 기재하도록 하고, 1회 실시 총비용으로 기재토록 권장하여 전체 비용 파악이 용이하도록 했다.
의료기관 내의 비치 장소는 안내데스크나 접수창구로 지정해 안내판을 설치하게 하고, 홈페이지 첫화면에 배치하고 검색 기능을 제공하도록 하여 찾기 쉽도록 했다.
복지부는 올해 1월 심평원을 통한 상급종합병원 비급여 가격공개에 이어, 이번에 관련 지침을 개정함으로써 의료기관 비급여 정보에 대한 국민들의 알권리를 보장하고 편의성을 높일 수 있을 것이라고 밝혔다.
이번 지침 개정 작업은 상급종합병원협의회와 함께 추진했다.
개정된 지침은 상급종합병원을 대상으로 9월 1일부터 우선 시행하고, 올해 말까지 전체 병원, 의원급 의료기관까지 확대한다.
['제', '##멋', '##대로', '비', '##급', '##여', '진', '##료', '##비', ',', '비', '##교', '#

In [9]:
train_inputs = tokenize_sentences(tokenizer, train_data, 450)
train_labels
validation_inputs = tokenize_sentences(tokenizer, validation_data, 450)
validation_labels
# print(train_inputs.shape, train_labels.shape, validation_inputs.shape, validation_labels.shape)

array([0, 1, 0, ..., 0, 0, 1])

In [10]:
print(train_inputs.input_ids.size())
print(train_inputs.attention_mask.size())
train_labels = torch.tensor(train_labels)
print(train_labels.size())

torch.Size([286317, 450])
torch.Size([286317, 450])
torch.Size([286317])


In [11]:
BATCH_SIZE = 8

train_data = TensorDataset(train_inputs.input_ids, train_inputs.attention_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_labels = torch.tensor(validation_labels)
validation_data = TensorDataset(validation_inputs.input_ids, validation_inputs.attention_mask, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

### 모델 생성

In [12]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB


In [13]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [14]:
# optimizer, learning rate scheduler 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8
                )

epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# metrics 설정
def compute_metrics(preds, labels):
    pred = np.argmax(preds, axis=1).flatten()

    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds)

    return {
        'accuracy': acc,
        'precision': precision,
    }

/home/aiffelmahimahi/anaconda3/envs/mahimahi/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):

    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

### 모델 학습

In [16]:
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [17]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [18]:
model.zero_grad()

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        loss = outputs[0]
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        model.zero_grad()

    avg_df_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average Data loss: {0:.2f}".format(avg_df_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of  35,790.    Elapsed: 0:01:51.
  Batch 1,000  of  35,790.    Elapsed: 0:03:42.
  Batch 1,500  of  35,790.    Elapsed: 0:05:33.
  Batch 2,000  of  35,790.    Elapsed: 0:07:24.
  Batch 2,500  of  35,790.    Elapsed: 0:09:14.
  Batch 3,000  of  35,790.    Elapsed: 0:11:05.
  Batch 3,500  of  35,790.    Elapsed: 0:12:56.
  Batch 4,000  of  35,790.    Elapsed: 0:14:47.
  Batch 4,500  of  35,790.    Elapsed: 0:16:37.
  Batch 5,000  of  35,790.    Elapsed: 0:18:28.
  Batch 5,500  of  35,790.    Elapsed: 0:20:19.
  Batch 6,000  of  35,790.    Elapsed: 0:22:10.
  Batch 6,500  of  35,790.    Elapsed: 0:24:00.
  Batch 7,000  of  35,790.    Elapsed: 0:25:51.
  Batch 7,500  of  35,790.    Elapsed: 0:27:42.
  Batch 8,000  of  35,790.    Elapsed: 0:29:33.
  Batch 8,500  of  35,790.    Elapsed: 0:31:23.
  Batch 9,000  of  35,790.    Elapsed: 0:33:14.
  Batch 9,500  of  35,790.    Elapsed: 0:35:05.
  Batch 10,000  of  35,790.    Elapsed: 0:36:

KeyboardInterrupt: 